In [1]:
from cProfile import label
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from scipy.signal import savgol_filter
from scipy import interpolate
import scipy
from mpl_toolkits.mplot3d import axes3d

In [5]:
import pinocchio as pin
model_pin = pin.buildModelFromUrdf("panda-model/panda_arm.urdf")
data_pin = model_pin.createData()

# Load dataset
data = np.loadtxt("data_ff.txt").T

time = data[0]
time = (time - time[0])*1e-6
position = data[1:8, :]
control_torque = data[8:15, :]
desired_acceleration = data[15:22, :]

#  Estimate acceleration and velocity from regularly space time grid
f_position = interpolate.interp1d(time, position, axis=1)
time_grid = np.linspace(time[0], time[-1], time.shape[0])

acceleration = savgol_filter(f_position(time_grid), window_length=111, polyorder=3, deriv=2, delta=np.mean(np.diff(time_grid)) )
f_acceleration = interpolate.interp1d(time_grid, acceleration, axis=1)

velocity = savgol_filter(f_position(time_grid), window_length=111, polyorder=2, deriv=1, delta=np.mean(np.diff(time_grid)) )
f_velocity = interpolate.interp1d(time_grid, velocity, axis=1)


# Resample dataset at lower frequency and regular spaced time grid
f_control_torque = interpolate.interp1d(time, control_torque, axis=1)
f_desired_acceleration = interpolate.interp1d(time, desired_acceleration, axis=1)

dT = 50e-3
time = np.arange(time[0], time[-1], dT)

position = f_position(time)
velocity = f_velocity(time)
acceleration = f_acceleration(time)
control_torque = f_control_torque(time)
desired_acceleration = f_desired_acceleration(time)

# Estimate model torque from acceleration and mass matrix
# est_torque = np.zeros_like(control_torque)*np.nan
# est_acceleration = np.zeros_like(control_torque)*np.nan
# for i in range((est_torque.shape[1])):
#     new_mass = pin.crba(model_pin, data_pin, position[:,i])
#     est_torque[:, i] = new_mass@acceleration[:, i]
#     est_acceleration[:, i] = scipy.linalg.solve(new_mass, control_torque[:, i], assume_a='pos')
#     est_acceleration[:, i] = np.linalg.pinv(new_mass)@control_torque[:, i]


# Robot ranges for nicer plots
position_limit = np.array([[-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973], 
                          [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]])
velocity_limit = np.array([2.1750, 2.1750, 2.1750, 2.1750, 2.6100, 2.6100, 2.6100])
acceleration_limit = np.array([15.0, 7.5, 10.0, 12.5, 15.0, 20.0, 20.0])
torque_limit = np.array([87, 87, 87, 87, 12, 12, 12])


In [6]:
data.shape

(22, 3339)

In [7]:
fig = make_subplots(rows=7, cols=1, row_titles=["Joint torque 1", "Joint torque 2", "Joint torque 3", "Joint torque 4", "Joint torque 5", "Joint torque 6", "Joint torque 7"])
for i in range(7):

    # Plot both models of gravity torque
    fig.add_trace(
    go.Scatter(x=time, y=desired_acceleration[i, :], line_color="orange", showlegend=False),
    row=i+1, col=1 )

    fig.add_trace(
    go.Scatter(x=time, y=acceleration[i, :], line_color="blue", showlegend=False),
    row=i+1, col=1 )

fig.update_layout(height=2000, width=1000, 
                title_text="Joint torques [N.m] from libfranka and urdf models, and applied torque <br>as a function of time [s]")
# fig.update(layout_showlegend=False)
fig.show()